In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [7]:
from fastai.imports import *
from fastai.plots import *
from torchtext.data import Field
from fastai.lm_rnn import seq2seq_reg
from quicknlp import SpacyTokenizer, print_batch, S2SModelData
from pathlib import Path

In [3]:
import matplotlib.pyplot as plt
import numpy as np

In [8]:
DATAPATH = "dataset/translation"


In [10]:
!cd $DATAPATH; ls

fra.txt train   valid


In [5]:
INIT_TOKEN = "<sos>"
EOS_TOKEN = "<eos>"
fields = [
    ("english", Field(init_token=INIT_TOKEN, eos_token=EOS_TOKEN, tokenize=SpacyTokenizer('en'), lower=True)),
    ("french", Field(init_token=INIT_TOKEN, eos_token=EOS_TOKEN, tokenize=SpacyTokenizer('fr'), lower=True))

]
batch_size = 64
data = S2SModelData.from_text_files(path=DATAPATH, fields=fields,
                                    train="train",
                                    validation="validation",
                                    source_names=["english", "french"],
                                    target_names=["french"],
                                    bs= batch_size
                                   )




OSError: Can't find model 'fr'

In [6]:
print(f'num tr batches: {len(data.trn_dl)}, num tr samples: {len(data.trn_ds)}')
print(f'num val batches: {len(data.val_dl)},num val samples: {len(data.val_ds)}')

num tr batches: 4, num tr samples: 201
num val batches: 4,num val samples: 201


In [24]:
emb_size = 300
nh = 1024
nl = 3
tnh = 512
learner = data.get_model(emb_sz=emb_size,
                         nhid=nh,
                         nlayers=nl,
                         bidir=True,
                         max_iterations=30,
                         att_nhid=tnh,
                         attention=True
                         )
reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
clip = 0.3
learner.reg_fn = reg_fn
learner.clip = clip

In [17]:
learner.summary()

Seq2SeqAttention(
  (encoder): RNNEncoder(
    (encoder): Embedding(102, 300, padding_idx=1)
    (encoder_with_dropout): EmbeddingDropout(
      (embed): Embedding(102, 300, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): Cell(
        (cell): WeightDrop(
          (module): LSTM(300, 512, dropout=0.3, bidirectional=True)
        )
      )
      (1): Cell(
        (cell): WeightDrop(
          (module): LSTM(1024, 512, dropout=0.3, bidirectional=True)
        )
      )
      (2): Cell(
        (cell): WeightDrop(
          (module): LSTM(1024, 150, dropout=0.3, bidirectional=True)
        )
      )
    )
    (dropouti): LockedDropout(
    )
    (dropouths): ModuleList(
      (0): LockedDropout(
      )
      (1): LockedDropout(
      )
      (2): LockedDropout(
      )
    )
  )
  (decoder): RNNAttentionDecoder(
    (encoder): Embedding(209, 300, padding_idx=1)
    (encoder_with_dropout): EmbeddingDropout(
      (embed): Embedding(209, 300, padding_idx=1)
    )
    (rnns): Modu

In [13]:
print_batch(lr=learner,dt=data, input_field="english", output_field="french",num_sentences=4)

batch: 0 sample : 0
input: <sos> join us . <eos> <pad>
target: joignez - vous à nous . <eos> <pad>
prediction: ['<sos> gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles gentilles g

In [12]:
learner.lr_find()